# Integrate Langchain + Gemini with Maxim tracing

In [1]:

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

## Initialize Maxim

In [2]:
from maxim import Maxim

logger = Maxim().logger()

[MaximSDK] Initializing Maxim AI(v3.8.3)
[MaximSDK] Using info logging level.
[MaximSDK] For debug logs, set global logging level to debug logging.basicConfig(level=logging.DEBUG).


## Invoke call with ChatVertexAI

In [3]:
from google.cloud import aiplatform

aiplatform.init(project="maxim-development-433105", location="us-west1")

In [4]:
import json
import uuid
from maxim.logger.langchain import MaximLangchainTracer
from langchain_core.tools import tool
from langchain_google_vertexai import VertexAI


# To use model
llm = VertexAI(model_name="gemini-2.0-flash-001")

langchain_tracer = MaximLangchainTracer(logger)

user_input = "latest football news"
system_message = "You are a helpful assistant."

trace_id = str(uuid.uuid4())
trace = logger.trace({"id": trace_id, "name": "langchain_trace"})
trace.end()
# Make the API call to Claude using LangChain
messages = [("system", system_message), ("human", user_input)]
response = llm.invoke(messages,config={"callbacks":[langchain_tracer],"metadata":{"maxim":{"trace_id":trace.id}}})
print(response)


# service 2
trace = logger.trace({"id": trace_id, "name": "langchain_trace"})
span_id=str(uuid.uuid4())
span = trace.span({"id": span_id,"name": "langchain_span"})

response = llm.invoke(messages,config={"callbacks":[langchain_tracer],"metadata":{"maxim":{"span_id":span_id}}})
span.event(str(uuid.uuid4()),"langchain_event")
span.end()


response generations=[[GenerationChunk(text="Okay! To give you the latest football news, I need to know what you're most interested in. Tell me, what kind of football news are you looking for? For example:\n\n*   **Which league/competition are you interested in?** (e.g., Premier League, Champions League, La Liga, NFL, etc.)\n*   **Which team are you interested in?** (e.g., Manchester United, Real Madrid, etc.)\n*   **What kind of news are you looking for?** (e.g., transfer rumors, match results, injury updates, manager news, etc.)\n\nOnce I have a better idea of what you're looking for, I can provide you with the most relevant information.\n", generation_info={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 14, 'candidates_token_count': 154, 'total_token_count': 168, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 14}], 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 154}]}})]] llm_output=None run=None {'run_id': UUID(

## With Chain

In [7]:
import time
import json

from langchain_google_vertexai import VertexAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate


llm = VertexAI(model="gemini-2.5-flash-preview-05-20",streaming=True,stream_usage=True)

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."), ("human", "{input}")]
)

# Create a chain using pipe
chain = prompt | llm | JsonOutputParser()

# Run the chain with callbacks for tracing
user_input = (
    "give me a basic json response. dont add anything else, just the json. not even ```"
)
response = ""
chunk_index = 0
start_time = time.time()
for event in chain.stream(
    input=user_input,
    config={
        "callbacks": [langchain_tracer],
        "metadata":{"maxim":{"trace_tags": {"test":"test-123"}}}
    },
):
    if chunk_index == 0:
        time_to_first_chunk = time.time() - start_time
    response = json.dumps(event)
    last_chunk = json.dumps(event)
    chunk_index += 1
time_to_last_chunk = time.time() - start_time
logger.flush()
print(response)

response generations=[[GenerationChunk(text='{}', generation_info={'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 30, 'candidates_token_count': 1, 'total_token_count': 73, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 30}], 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 1}], 'thoughts_token_count': 42}})]] llm_output=None run=None {'run_id': UUID('312589d3-8f21-43ff-b2c0-5ae964a74d1e'), 'parent_run_id': UUID('3e878b31-9e25-48b1-abbf-605f4e2b203d'), 'tags': ['seq:step:2']}
{}
